![img](https://www.uib.no/sites/w3.uib.no/files/styles/content_main_wide_1x/public/media/corona_0.jpg?itok=nY-uRO-n&timestamp=1582750728)

Last Updated On

In [1]:
from datetime import datetime
print(datetime.now().strftime('%d-%m-%Y %I:%M %p'))

25-05-2020 02:53 PM


## [For Live Updates Open This Notebook in Colab](https://colab.research.google.com/github/prasadpatil99/Web-Scraping/blob/master/Web-Scraping-Covid-19-Cases/LIVE-Covid-19-Updates-Of-India-&-Other-Countries.ipynb)

## Data Gathering

**For cases across the world**

In [2]:
import numpy as np
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings("ignore")

In [3]:
wd_url = "https://www.worldometers.info/coronavirus/"
req= requests.get(wd_url)
countrylist = pd.read_html(req.text)
df_country = countrylist[0]
country = df_country['Country,Other'].tolist()
totalcases = df_country['TotalCases'].tolist()
Newcases = df_country['NewCases'].tolist()
deaths = df_country['TotalDeaths'].tolist()
newdeaths = df_country['NewDeaths'].tolist()
totalrecovered = df_country['TotalRecovered'].tolist()

In [4]:
df1 = pd.DataFrame({'Country':country, 'TotalCases':totalcases, 'NewCases':Newcases, 
                    'TotalDeaths':deaths, 'NewDeaths':newdeaths, 'TotalRecovered':totalrecovered})
df = df1.loc[1:(len(df1)-2)]
df.head()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered
1,USA,1686436,NaN,99300.0,NaN,451702.0
2,Brazil,365213,"+1,595",22746.0,30.0,149911.0
3,Russia,353427,"+8,946",3633.0,92.0,118798.0
4,Spain,282852,NaN,28752.0,NaN,196958.0
5,UK,259559,NaN,36793.0,NaN,NaN


In [5]:
df_world = df1.loc[:0]

Removing symbols and replacing null values from dataframe

In [6]:
def null_values(data):
    data.fillna(0, inplace=True)
    return data
df = null_values(df)

In [7]:
def preprocess(data):
    for cols in data:
        data[cols] = data[cols].map(lambda x: str(x).replace('+',''))
        data[cols] = data[cols].map(lambda x: str(x).replace(',',''))
preprocess(df_world)
preprocess(df)

**For cases in India**

In [8]:
res = requests.get("https://www.mohfw.gov.in")
soup = BeautifulSoup(res.content,'lxml') 
table = soup.find_all('table')[0]
df_Ind = pd.read_html(str(table))[0]
state = df_Ind["Name of State / UT"].tolist()
cases = df_Ind["Total Confirmed cases*"].tolist()
cured = df_Ind["Cured/Discharged/Migrated"].tolist()
deaths = df_Ind["Deaths**"].tolist()

In [9]:
df_Ind = pd.DataFrame({'States':state, 'Cases':cases, 'Cured':cured, 'Death':deaths})
df_Indtotal = df_Ind.loc[[35]]
df_Ind = df_Ind.loc[:(len(df_Ind)-7)]
df_Ind.head()

,States,Cases,Cured,Death
0,Andaman and Nicobar Islands,33,33,0
1,Andhra Pradesh,2823,1856,56
2,Arunachal Pradesh,1,1,0
3,Assam,378,55,4
4,Bihar,2587,702,13


In [10]:
import pandas as pd 
import plotly.express as px

import cufflinks
from plotly.offline import iplot
cufflinks.go_offline()

## Data Visualisation - World

In [11]:
df_world.head()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered
0,World,5516842,22387,346949.0,515.0,2310215.0


In [12]:
df_world.mean().iplot(kind='bar',title="World's Total Cases, New Cases, Total Deaths, New Deaths & Total Recovered")

In [13]:
df_newcases = df.copy()
df_newcases["NewCases"] = pd.to_numeric(df_newcases["NewCases"])
df_newcases["NewDeaths"] = pd.to_numeric(df_newcases["NewDeaths"])
df_newcases=df_newcases.sort_values(by = ["NewCases"], ascending = False)
df_newcases = df_newcases[(df_newcases['NewCases']>0) & (df_newcases['NewDeaths']>0)]

fig = px.bar(df_newcases, x='Country', y ='NewCases', color='Country',text='NewCases', height=500, width=1500, 
             title ="Today's New Cases Across Countries")
fig.show()

In [14]:
df["TotalRecovered"] = pd.to_numeric(df["TotalRecovered"])
df=df.sort_values(by = ["TotalRecovered"], ascending = False).head(50)

df.iplot(
    x='Country',
    y=['TotalRecovered'],
    mode='lines',
    size=8,
    title="Top 50 Countries With Recovered Cases")

In [15]:
df_newcases.iplot(
    x='Country',
    y=['NewDeaths','NewCases'],
    mode='lines',
    size=8,
    title="Today's New Cases & New Deaths")

In [16]:
df_totaldeaths = df.copy()
df_totaldeaths["TotalDeaths"] = pd.to_numeric(df_totaldeaths["TotalDeaths"])
df_totaldeaths = df_totaldeaths[df_totaldeaths['TotalDeaths']>150]
df_totaldeaths=df_totaldeaths.sort_values(["TotalDeaths"], axis=0, ascending=[False])
fig = px.bar(df_totaldeaths, x='Country', y = 'TotalDeaths', color='Country', text='TotalDeaths', height=500, 
             width=1200,title ='Number of Total Deaths Across Countries Greater Than 150')
fig.show()

In [17]:
fig = px.pie(df, values='TotalCases',names='Country', labels={'cured':'TotalCases'}, hover_data=['TotalCases'],
            title='Total Number of Cases Countries Suffered')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## Data Visualisation - India

In [18]:
import plotly.io as pio
pio.templates.default = "plotly_dark"

df_Indtotal.head()

,States,Cases,Cured,Death
35,Total#,138845,57721,4021


In [19]:
df_Indtotal.mean().iplot(kind='bar',
                        layout=dict(title="India's Total Cases, Cured Cases & Total Deaths"))

In [20]:
df_Indcases = df_Ind.copy()
df_Indcases["Cases"] = pd.to_numeric(df_Indcases["Cases"])
df_Indcases=df_Indcases.sort_values(by = ["Cases"], ascending = False)
fig = px.bar(df_Indcases, x='States', y ='Cases', color='States',text='Cases', height=500, width=950, 
             title ='Number of Total Cases in India')
fig.show()

In [21]:
df_Indcured = df_Ind.copy()
df_Indcured["Cured"] = pd.to_numeric(df_Indcured["Cured"])
df_Indcured=df_Indcured.sort_values(by = ["Cured"], ascending = False)
fig = px.bar(df_Indcured, x='States', y ='Cured', color='States',text='Cured', height=500, width=950, 
             title ='Total Cured Cases in Country')
fig.show()

In [22]:
df_Indcured = df_Ind.copy()
df_Indcured["Cases"] = pd.to_numeric(df_Indcured["Cases"])
df_Indcured=df_Indcured.sort_values(by = ["Cases"], ascending = False)
df_Indcured.iplot(
    x='States',
    y=['Cured','Cases'],
    mode='lines',
    size=8,
    layout=dict(title='Rate of Total Cured & Total Cases In India'))

In [23]:
fig = px.pie(df_Indcured, values='Cases',names='States', labels={'cured':'Cases'}, hover_data=['Cases'],
            title="Total Cases India Suffered as per States")
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## References
https://plotly.com/python/<br>
https://www.mohfw.gov.in <br>
https://www.worldometers.info/coronavirus